In [ ]:
# from google.colab import drive
# # drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM,GRU
from keras.preprocessing import sequence
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score    
from tensorflow.python.keras import models, layers, optimizers   
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, SpatialDropout1D
from keras.layers.wrappers import Bidirectional
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
df = pd.read_csv("data/train.csv")
df.head()

,3,more like funchuck,"Gave this to my dad for a gag gift after directing ""Nunsense,"" he got a reall kick out of it!"
0,5,Inspiring,I hope a lot of people hear this cd. We need m...
1,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,5,Too good to be true,Probably the greatest soundtrack in history! U...
4,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."


In [3]:
df.columns=["rating","title","text"]
df

,rating,title,text
0,5,Inspiring,I hope a lot of people hear this cd. We need m...
1,5,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,4,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,5,Too good to be true,Probably the greatest soundtrack in history! U...
4,5,There's a reason for the price,"There's a reason this CD is so expensive, even..."
...,...,...,...
2999994,1,Don't do it!!,The high chair looks great when it first comes...
2999995,2,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
2999996,2,"compact, but hard to clean","We have a small house, and really wanted two o..."
2999997,3,Hard to clean!,I agree with everyone else who says this chair...


In [4]:
train=df

import numpy as np # linear algebra
import pandas as pd 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/lupin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
for i in range(len(train)):
  if train['rating'][i]==1 or train['rating'][i]==2:
    train['rating'][i]=0
  else:
    train['rating'][i]=1

/tmp/ipykernel_6850/2925514527.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['rating'][i]=1
/tmp/ipykernel_6850/2925514527.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['rating'][i]=0


In [6]:
train

,rating,title,text
0,1,Inspiring,I hope a lot of people hear this cd. We need m...
1,1,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,1,Chrono Cross OST,The music of Yasunori Misuda is without questi...
3,1,Too good to be true,Probably the greatest soundtrack in history! U...
4,1,There's a reason for the price,"There's a reason this CD is so expensive, even..."
...,...,...,...
2999994,0,Don't do it!!,The high chair looks great when it first comes...
2999995,0,"Looks nice, low functionality",I have used this highchair for 2 kids now and ...
2999996,0,"compact, but hard to clean","We have a small house, and really wanted two o..."
2999997,1,Hard to clean!,I agree with everyone else who says this chair...


In [7]:
import re
import string
from nltk.corpus import stopwords


In [8]:
df_test = pd.read_csv("data/test.csv")
df_test.head()

,1,mens ultrasheer,"This model may be ok for sedentary types, but I'm active and get around alot in my job - consistently found these stockings rolled up down by my ankles! Not Good!! Solution: go with the standard compression stocking, 20-30, stock #114622. Excellent support, stays up and gives me what I need. Both pair of these also tore as I struggled to pull them up all the time. Good riddance/bad investment!"
0,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
1,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
2,2,Oh dear,I was excited to find a book ostensibly about ...
3,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."
4,2,Incorrect Disc,"I love the style of this, but after a couple y..."


In [9]:
df_test.columns=["rating","title","text"]
df_test

,rating,title,text
0,4,Surprisingly delightful,This is a fast read filled with unexpected hum...
1,2,"Works, but not as advertised",I bought one of these chargers..the instructio...
2,2,Oh dear,I was excited to find a book ostensibly about ...
3,2,Incorrect disc!,"I am a big JVC fan, but I do not like this mod..."
4,2,Incorrect Disc,"I love the style of this, but after a couple y..."
...,...,...,...
649994,5,Pretty Cool!,We got it for our mom's birthday. She LOVES it...
649995,5,great cd,"this cd is very good. i especially love ""cats ..."
649996,2,An interesting look into Boston's comedy clubs,This was a good documentary on the history of ...
649997,5,Du vol...pour les cowboys!,Avez-vous déjà vu un CD double et un DVD avec ...


In [10]:
test=df_test

In [11]:
def url(text):
     url=re.compile(r"https?://\S+|www\.\S+")
     return url.sub(r" ",text)
train['text']=train.text.map(lambda x:url(x))
test['text']=test.text.map(lambda x:url(x))

In [12]:
def html(text):
  cleaned_data = re.compile('<.*?>')
  return cleaned_data.sub(r" ",text)
train['text']=train.text.map(lambda x:html(x))
test['text']=test.text.map(lambda x:html(x))

In [13]:
def punctuation(text):
   data=str.maketrans(' ',' ',string.punctuation)
   return text.translate(data)
train['text']=train.text.map(lambda x:punctuation(x))
test['text']=test.text.map(lambda x:punctuation(x))

In [14]:
def num(texts):
   data = re.sub(r'\d+', '', texts)
   return data
train['text']=train['text'].map(num)
test['text']=test['text'].map(num)

In [15]:
stop_words=set(stopwords.words("english"))
 
def stopword(data):
   data=[word.lower() for word in data.split() if word.lower() not in stop_words]
   return " ".join(data)
train['text']=train['text'].map(stopword)
test['text']=test['text'].map(stopword)

In [16]:
for i in range(len(test)):
  if test['rating'][i]==1 or test['rating'][i]==2:
    test['rating'][i]=0
  else:
    test['rating'][i]=1

/tmp/ipykernel_6850/4053600729.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['rating'][i]=1
/tmp/ipykernel_6850/4053600729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['rating'][i]=0


In [17]:
test

,rating,title,text
0,1,Surprisingly delightful,fast read filled unexpected humour profound in...
1,0,"Works, but not as advertised",bought one chargersthe instructions say lights...
2,0,Oh dear,excited find book ostensibly muslim feminism v...
3,0,Incorrect disc!,big jvc fan like model suspiscious saw several...
4,0,Incorrect Disc,love style couple years dvd giving problems do...
...,...,...,...
649994,1,Pretty Cool!,got moms birthday loves favorites weather chan...
649995,1,great cd,cd good especially love cats mars would highly...
649996,0,An interesting look into Boston's comedy clubs,good documentary history two local comedy club...
649997,1,Du vol...pour les cowboys!,avezvous déjà vu un cd double et un dvd avec p...


In [18]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
def Stemming(text):
   stem=[]
   stopword = stopwords.words('english')
   snowball_stemmer = SnowballStemmer('english')
   word_tokens = nltk.word_tokenize(text)
   stemmed_word = [snowball_stemmer.stem(word) for word in word_tokens]
   stem=' '.join(stemmed_word)
   return stem

[nltk_data] Downloading package punkt to /home/lupin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
train['text']=train['text'].map(Stemming)
test['text']=test['text'].map(Stemming)

In [20]:
tokenized = Tokenizer(num_words=32000,oov_token="<OOV>")
tokenized.fit_on_texts(train['text'])

word_index = tokenized.word_index

training_sequences = tokenized.texts_to_sequences(train['text'])
training_padded = pad_sequences(training_sequences, maxlen=100, padding="post", truncating="post")
testing_sequences = tokenized.texts_to_sequences(test['text'])
testing_padded = pad_sequences(testing_sequences, maxlen=100, padding="post", truncating="post")

In [21]:
model_lstm = Sequential()
model_lstm.add(Embedding(32000, 64, input_length=100))
model_lstm.add(SpatialDropout1D(0.2))
model_lstm.add(LSTM(256, return_sequences=True, dropout=0.2))
model_lstm.add(LSTM(256, return_sequences=False,dropout=0.2))
model_lstm.add(Dense(1))
model_lstm.add(Activation('sigmoid'))
model_lstm.summary()

2022-04-27 01:51:58.057158: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-27 01:51:58.728743: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10018 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:41:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 64)           2048000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 64)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100, 256)          328704    
                                                                 
 lstm_1 (LSTM)               (None, 256)               525312    
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                        

In [22]:
model_bilstm = Sequential()
model_bilstm.add(Embedding(32000, 64, input_length=100))
model_bilstm.add(SpatialDropout1D(0.2))
model_bilstm.add(Bidirectional(LSTM(256, dropout=0.2)))
model_bilstm.add(Dense(1))
model_bilstm.add(Activation('sigmoid'))
model_bilstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 64)           2048000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 100, 64)          0         
 lDropout1D)                                                     
                                                                 
 bidirectional (Bidirectiona  (None, 512)              657408    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
 activation_1 (Activation)   (None, 1)                 0         
                                                                 
Total params: 2,705,921
Trainable params: 2,705,921
No

In [23]:
adam_lstm=Adam(lr=0.000001)
model_lstm.compile(loss='binary_crossentropy',optimizer=adam_lstm,metrics=['accuracy'] )
adam_bilstm=Adam(lr=0.0001)
model_bilstm.compile(loss='binary_crossentropy',optimizer=adam_bilstm,metrics=['accuracy'] )

/home/lupin/gilbert/venv/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [24]:
history=model_lstm.fit(training_padded,train['rating'], epochs=1, batch_size=128,verbose = 1,callbacks = [EarlyStopping(monitor='val_accuracy', patience=2)],validation_data=(testing_padded,test['rating']))

2022-04-27 01:52:04.629617: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


23438/23438 [==============================] - 19102s 815ms/step - loss: 0.6738 - accuracy: 0.6000 - val_loss: 0.6731 - val_accuracy: 0.6000


In [25]:
y_pred = model_lstm.predict(testing_padded)
y_pred = (y_pred > 0.5)
X_test=testing_padded
y_test=test['rating']

In [26]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix
final_Acc = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % final_Acc)

precision_final = precision_score(y_test, y_pred)
print('Precision: %f' % precision_final)

recall_final = recall_score(y_test, y_pred)
print('Recall: %f' % recall_final)

f1_final = f1_score(y_test, y_pred)
print('F1 score: %f' % f1_final)
 

auc = roc_auc_score(y_test, y_pred)

matrix = confusion_matrix(y_test, y_pred)
print(matrix)

Accuracy: 0.600001
Precision: 0.600001
Recall: 1.000000
F1 score: 0.750001
[[     0 259999]
 [     0 390000]]


In [27]:
history=model_bilstm.fit(training_padded,train['rating'], epochs=1, batch_size=128,verbose = 1,callbacks = [EarlyStopping(monitor='val_accuracy', patience=2)],validation_data=(testing_padded,test['rating']))

23438/23438 [==============================] - 18661s 796ms/step - loss: 0.4361 - accuracy: 0.7943 - val_loss: 0.4087 - val_accuracy: 0.8095


In [28]:
y_pred = model_bilstm.predict(testing_padded)
y_pred = (y_pred > 0.5)
X_test=testing_padded
y_test=test['rating']

In [29]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix
final_Acc = accuracy_score(y_test, y_pred)
print('Accuracy: %f' % final_Acc)

precision_final = precision_score(y_test, y_pred)
print('Precision: %f' % precision_final)

recall_final = recall_score(y_test, y_pred)
print('Recall: %f' % recall_final)

f1_final = f1_score(y_test, y_pred)
print('F1 score: %f' % f1_final)
 

auc = roc_auc_score(y_test, y_pred)

matrix = confusion_matrix(y_test, y_pred)

Accuracy: 0.809514
Precision: 0.835694
Recall: 0.849554
F1 score: 0.842567


In [30]:
model_bilstm.save('bilstm.h5')
model_lstm.save('lstm.h5')